In [1]:
import os
from os import path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plot
import seaborn as sbn
sbn.set()
%matplotlib inline

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.covariance import (EllipticEnvelope, EmpiricalCovariance)
from sklearn.neighbors import LocalOutlierFactor 
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

In [5]:
curr_dir = os.getcwd()
df_telemetry_data = pd.read_csv(path.join(curr_dir,'data/telemetries.csv'),parse_dates=['measurement_time'])
df_telemetry_data.head()

,container_id,measurement_time,product_id,temperature,target_temperature,ambiant_temperature,kilowatts,time_door_open,content_type,defrost_cycle,oxygen_level,nitrogen_level,humidity_level,carbon_dioxide_level,vent_1,vent_2,vent_3,maintenance_required
0,C01,2019-10-21 21:58:47.778469,P02,6.421550,6,20.875069,5.104784,-0.849733,2,6,22.582955,78.414413,59.751720,2.297364,True,True,True,0
1,C01,2019-10-21 22:03:47.778469,P02,5.856868,6,21.128387,5.122725,0.617787,2,5,21.501824,78.449830,61.570283,3.782031,True,True,True,0
2,C01,2019-10-21 22:08:47.778469,P02,5.656720,6,21.171572,2.829637,2.119314,2,6,20.104576,77.771802,61.443443,5.868899,True,True,True,0
3,C01,2019-10-21 22:13:47.778469,P02,6.053072,6,20.665113,1.987387,1.103350,2,4,19.039297,75.704534,60.300971,4.518827,True,True,True,0
4,C01,2019-10-21 22:18:47.778469,P02,6.409696,6,21.025121,2.898705,0.861419,2,5,21.075411,78.685539,59.725525,4.676924,True,True,True,0


In [6]:
df_telemetry_data.columns

Index(['container_id', 'measurement_time', 'product_id', 'temperature',
       'target_temperature', 'ambiant_temperature', 'kilowatts',
       'time_door_open', 'content_type', 'defrost_cycle', 'oxygen_level',
       'nitrogen_level', 'humidity_level', 'carbon_dioxide_level', 'vent_1',
       'vent_2', 'vent_3', 'maintenance_required'],
      dtype='object')

In [7]:
df_telemetry_data.dtypes

container_id                    object
measurement_time        datetime64[ns]
product_id                      object
temperature                    float64
target_temperature               int64
ambiant_temperature            float64
kilowatts                      float64
time_door_open                 float64
content_type                     int64
defrost_cycle                    int64
oxygen_level                   float64
nitrogen_level                 float64
humidity_level                 float64
carbon_dioxide_level           float64
vent_1                            bool
vent_2                            bool
vent_3                            bool
maintenance_required             int64
dtype: object

In [8]:
df_telemetry_data['defrost_cycle'].value_counts()

3    2352
4    2265
5    2196
6    2187
Name: defrost_cycle, dtype: int64

In [9]:
df_telemetry_data_training = df_telemetry_data[[ 'temperature','target_temperature', 'ambiant_temperature', 'kilowatts', \
       'time_door_open', 'defrost_cycle', 'oxygen_level', 'nitrogen_level', 'humidity_level', 'carbon_dioxide_level']]
df_telemetry_data_training['temperature_difference'] = df_telemetry_data_training['target_temperature'] - \
                                                                                                     df_telemetry_data_training['temperature']
df_telemetry_data_training['temperature_ambient_ratio'] = df_telemetry_data_training['ambiant_temperature']/df_telemetry_data_training['temperature']
df_telemetry_data_training.drop(['temperature','target_temperature', 'ambiant_temperature'], axis=1, inplace=True)
print(df_telemetry_data_training.shape)
df_telemetry_data_training.head()

(9000, 9)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/i

,kilowatts,time_door_open,defrost_cycle,oxygen_level,nitrogen_level,humidity_level,carbon_dioxide_level,temperature_difference,temperature_ambient_ratio
0,5.104784,-0.849733,6,22.582955,78.414413,59.751720,2.297364,-0.421550,3.250784
1,5.122725,0.617787,5,21.501824,78.449830,61.570283,3.782031,0.143132,3.607455
2,2.829637,2.119314,6,20.104576,77.771802,61.443443,5.868899,0.343280,3.742729
3,1.987387,1.103350,4,19.039297,75.704534,60.300971,4.518827,-0.053072,3.413988
4,2.898705,0.861419,5,21.075411,78.685539,59.725525,4.676924,-0.409696,3.280206


In [10]:
ss = StandardScaler()
mms = MinMaxScaler()
rs = RobustScaler()
ocs = OneClassSVM()
isf = IsolationForest(n_estimators=100)
ee = EllipticEnvelope()
ec = EmpiricalCovariance()
lof = LocalOutlierFactor(novelty=True) # different from anomaly detection, but can be used to interpret the anomaly

#### Isolation Forest 

In [11]:
isf.fit(df_telemetry_data_training)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


IsolationForest(behaviour='old', bootstrap=False, contamination='legacy',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [12]:
isf.predict(df_telemetry_data_training)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


array([-1,  1,  1, ...,  1,  1, -1])

In [13]:
isf.score_samples(df_telemetry_data_training)

array([-0.51263385, -0.46045511, -0.44751288, ..., -0.41780285,
       -0.40040959, -0.52225286])

#### Elliptic Envelope
Make sure to check the underlying distribution before using this. Make sure it is a guassian distribution underneath, or project it.

In [14]:
ocs.fit(df_telemetry_data_training)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto_deprecated',
            kernel='rbf', max_iter=-1, nu=0.5, random_state=None,
            shrinking=True, tol=0.001, verbose=False)

In [15]:
ocs.predict(df_telemetry_data_training)

array([-1, -1, -1, ...,  1,  1, -1])

In [16]:
ocs.score_samples(df_telemetry_data_training)

array([305.53504339, 571.98447809, 537.11605012, ..., 784.16205545,
       833.80305604, 380.17798776])

#### Local Outlier Factor

In [17]:
lof.fit(df_telemetry_data_training)

/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=20, novelty=True, p=2)

In [18]:
lof.score_samples(df_telemetry_data_training) # This is incorrect because you need to provide new data, as opposed to training data for novelty detection.

array([-1.19405577, -1.07520773, -1.08883455, ..., -1.01537208,
       -0.97374384, -1.14485316])